In [1]:
import datetime as dt
import pandas as pd
import psycopg2
import pytz
import re
import sys
import time
import warnings
from bs4 import BeautifulSoup
from io import StringIO
from psycopg2.extras import execute_values
from psycopg2.extensions import AsIs
from typing import List
from urllib.request import urlopen
import sys
sys.path.insert(0, '../..')
from db_utils import generate_unique_game_id, execute_sql_script
from scrape_all_games_data import get_daily_games_at_date, clean_df, insert_daily_games_at_date, backfill_games
from typing import Callable
from db_utils import load_config, retrieve_data

In [2]:
backfill_games()

Backfilling games around 2014-01-07
Successfully logged cfb.all_games_log with 2014-01-07 - no data for this day
Successfully logged cfb.all_games_log with 2014-01-08 - no data for this day
Backfilling games around 2014-01-10
Successfully logged cfb.all_games_log with 2014-01-10 - no data for this day
Successfully logged cfb.all_games_log with 2014-01-09 - no data for this day
Successfully logged cfb.all_games_log with 2014-01-11 - no data for this day
Backfilling games around 2014-01-13
Successfully logged cfb.all_games_log with 2014-01-13 - no data for this day
Successfully logged cfb.all_games_log with 2014-01-12 - no data for this day
Successfully logged cfb.all_games_log with 2014-01-14 - no data for this day
Backfilling games around 2014-08-15
Successfully logged cfb.all_games_log with 2014-08-15 - no data for this day
Successfully logged cfb.all_games_log with 2014-08-14 - no data for this day
Successfully logged cfb.all_games_log with 2014-08-16 - no data for this day
Backfilli

URLError: <urlopen error [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond>

In [ ]:
def get_rank_from_row(row):
    pattern2 = r"^(.*?)\s\((\d{1,2})\)$"
    university = row[0]
    match = re.fullmatch(pattern2, university)
    if match:
         return match.group(1), int(match.group(2))
    else:
         return university, None

In [ ]:
get_rank_from_row(all_games_table[5].iloc[2])

('Arizona', '21')

In [ ]:
all_games_table[5]

,0,1,2
0,Sat 8/31,Sat 8/31,Sat 8/31
1,New Mexico,39,Final
2,Arizona (21),61,NaN


In [ ]:
temp.iloc[0][0]

'Western Kentucky'

In [ ]:
regex2 = r".*\(\b([1-9]|1[0-9]|2[0-5])\b\)"
re.fullmatch(regex2, "Western Kentucky (5)")

<re.Match object; span=(0, 20), match='Western Kentucky (5)'>

In [ ]:
temp = all_games_table[3][1:]
temp.loc[:, "rank"] = None
# temp

/var/folders/z_/5kxjclhd5090czxnlnxslydr0000gn/T/ipykernel_88211/1111760236.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp.loc[:, "rank"] = None


regex = "^[A-Za-z]{3} \d{1,2}/\d{2}$"
string = all_games_table[3].iloc[0][0]

s = re.fullmatch(regex, string)

In [ ]:
s

<re.Match object; span=(0, 8), match='Sat 8/31'>

In [ ]:
month = month.lower()

    # URL to scrape, notice f string:
    url = (
        f"https://www.basketball-reference.com/leagues/NBA_%(year)s_games-%(month)s.html"
        % {"year": year, "month": month}
    )
    try:
        html = urlopen(url)
    except Exception as e:
        print(e)
        print(
            "Failure: %(month)s, %(year)s all_games URL not pulled"
            % {"month": month.capitalize(), "year": year}
        )
        return None

    # Kicks you out if you request over 20 times over a minute
    time.sleep(3.5)

    soup = BeautifulSoup(html, features="html.parser")

    # Convert the tables into a string and wrap it with StringIO
    tables = soup.find_all("table", {"id": re.compile("schedule")})
    html_string = "\n".join(str(table) for table in tables)
    html_io = StringIO(html_string)

    all_games_table = pd.read_html(html_io, header=0)[0]
    all_games_table = clean_all_games_table(all_games_table)
    if all_games_table is not None:
        all_games_table["Unique_ID"] = all_games_table.apply(
            generate_unique_game_id, axis=1
        )
    return all_games_table